# QUALITY CONTROL OF ADCP DATA

In [36]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys
import os
home_path = os.environ['HOME']
abs_dir = os.path.dirname(os.getcwd())
cwd_path = os.path.join(abs_dir, 'boias', 'bmo_br', 'real_time')
bd_path = os.path.join(abs_dir, 'boias', 'bmo_br', 'bd')


In [10]:
sys.path.append(cwd_path)
sys.path.append(bd_path)
from bmo_database import *
from bmo_adjust_data import *
import bmo_quality_control as bqc

In [28]:
conn = connect_database_remo('PRI')
buoy_id = bmo_on(conn)
id = buoy_id['buoy_id'][0]
last_date_raw = check_last_date(conn, 'bmo_br_current', id)
last_date_raw = last_date_raw[0][0]
time_interval = 24 # hours interval of data to be qualified
bmo_general = get_data_table_db(conn, id, last_date_raw, 'bmo_br_current','ALL')
bmo_general.set_index('date_time', inplace = True)
bmo_general.head()

,buoy_id,id,lat,lon,cspd1,cdir1,cspd2,cdir2,cspd3,cdir3,...,cspd16,cdir16,cspd17,cdir17,cspd18,cdir18,cspd19,cdir19,cspd20,cdir20
date_time,,,,,,,,,,,,,,,,,,,,,
2020-12-04 19:00:00,2,371,-22.8843,-43.1356,-9999.0,-9999,-9999.0,-9999,-9999.0,-9999,...,-9999.0,-9999,-9999.0,-9999,-9999.0,-9999,None,None,None,None
2020-12-08 09:00:00,2,372,-22.8976,-43.1522,-9999.0,-9999,-9999.0,-9999,-9999.0,-9999,...,-9999.0,-9999,-9999.0,-9999,-9999.0,-9999,None,None,None,None
2020-12-08 08:00:00,2,373,-22.8842,-43.1356,-9999.0,-9999,-9999.0,-9999,-9999.0,-9999,...,-9999.0,-9999,-9999.0,-9999,-9999.0,-9999,None,None,None,None
2020-12-08 07:00:00,2,374,-22.8842,-43.1356,-9999.0,-9999,-9999.0,-9999,-9999.0,-9999,...,-9999.0,-9999,-9999.0,-9999,-9999.0,-9999,None,None,None,None
2020-12-08 06:00:00,2,375,-22.8842,-43.1356,-9999.0,-9999,-9999.0,-9999,-9999.0,-9999,...,-9999.0,-9999,-9999.0,-9999,-9999.0,-9999,None,None,None,None


In [32]:
x = list(bmo_general.columns[4:-1])
x[0][0:4] == 'cspd'

True

In [40]:
df = bmo_general
def definition_flag_pandas_adcp(df):

    flag_data = df [list(df.columns[4:-1])] * 0

    flag_data = flag_data.replace(np.nan, 0).astype(int)

    return flag_data

flag_data = definition_flag_pandas_adcp(df)
# flag_data.head()
parameters = list(df.columns[4:-1])


In [41]:
for parameter in parameters:
    # missing value checks
    if parameter[0:4] == cspd:
        df[parameter] = df[parameter] * 1000
    flag_data = qc.mis_value_check(df, limits.mis_value_limits, flag_data, parameter)
    # coarse range check
    flag_data = qc.range_check(df, limits.range_limits, flag_data, parameter)
    # soft range check
    flag_data = qc.range_check_climate(df, limits.climate_limits, flag_data, parameter)

NameError: name 'cspd' is not defined